In [1]:
import numpy as np

In [19]:
def norma(x, p=2):
    x = np.ravel(x)
    return np.sum(np.abs(x)**p)**(1/p)

In [20]:
def multMat(A, x):
    A = np.array(A)
    x = np.array(x)
    res = np.zeros(A.shape[0])
    for i in range(A.shape[0]):
        tot = 0
        for k in range(A.shape[1]):
            tot += A[i, k] * x[k]
        res[i] = tot
    return res

def prodMat(A, B):
    if A.shape[1] != B.shape[0]:
        print("No se puede hacer el producto matricial")
        return
    res = np.zeros((A.shape[0], B.shape[1]))

    for i in range(res.shape[0]):
        for j in range(res.shape[1]):
            tot = 0
            for k in range(A.shape[1]):
                tot += A[i, k] * B[k, j]
            res[i, j] = tot

    return res

def prodPunto(v, w):
    return np.sum(v*w)

In [21]:
def aplicTrans(A, v):
    """
    Asume que el vector es columna
    """

    if A.shape[1] != v.shape[0]:
        print("No se puede aplicar la matriz al vector")
        return
    res = np.zeros((A.shape[0]))

    for i in range(res.shape[0]):
        tot = 0            
        for k in range(A.shape[1]):
            tot += A[i, k] * v[k]
        res[i] = tot

    return res

In [22]:
def metpot2k(A, tol=1e-15, K=1000):
    N = A.shape[0]

    # Caso matriz nula → todos los autovalores son 0
    if norma(A, 2) < tol:
        v = np.random.rand(N)
        v = v / norma(v, 2)
        v = np.expand_dims(v, axis=0).T
        return v, 0.0, 0

    # vector inicial
    v = np.random.rand(N)
    v = v / norma(v, 2)

    v_monio = aplicTrans(A, v)
    v_monio = v_monio / norma(v_monio, 2)

    e = prodPunto(v_monio, v)
    k = 0
    
    while abs(e - 1) > tol and k < K:

        v = v_monio

        # doble aplicación
        w = aplicTrans(A, v)
        w = w / norma(w, 2)

        v_monio = aplicTrans(A, w)
        v_monio = v_monio / norma(v_monio, 2)

        e = prodPunto(v_monio, v)
        k += 1

    v_monio = np.expand_dims(v_monio, axis=0).T
    # lambd = prodMat(prodMat(v_monio.T, A), v_monio)[0][0]
    lambd = v_monio.T@A@v_monio[0][0]

    return v_monio, lambd, e-1


In [2]:
def fullyConnectedSVD(X,Y):
    U, S, V = svd_reducida(X)
    
    # #convertir S a S+
    # for i in range (S.shape):
    #     S[i] = 1/S[i]
    
    VS= np.zeros(V.shape)
    r = np.zeros[1]
    # en vez de pasar a S+ divido cada Vi por S[i] que es como multiplicar por 1/S[i] (inversa de sigma)
    for i in range(r):
        VS[:, i] = V[:, i]/S[i]
    
    X_ps = VS@U.T
    W = Y@X_ps
    
    return W

In [ ]:
def es_svd_valida(A, U, S, V, tol=1e-8):
    # 1. Chequear que U es ortonormal
    if not np.allclose(U.T @ U, np.eye(U.shape[1]), atol=tol):
        print("U no es ortonormal")
        # return False
    
    # 2. Chequear que V es ortonormal
    if not np.allclose(V.T @ V, np.eye(V.shape[1]), atol=tol):
        print("V no es ortonormal")
        # return False
    
    # 3. Chequear que S es diagonal
    if not np.allclose(S, np.diag(np.diag(S)), atol=tol):
        print("S no es diagonal")
        return False

    # 4. Chequear reconstrucción
    A_rec = U @ S @ V.T
    if not np.allclose(A, A_rec, atol=tol):
        print("A != U S V^T")
        return False
    
    return True

In [ ]:
# Después de calcular U y V
print("Ortogonalidad U:", np.max(np.abs(U.T @ U - np.eye(U.shape[1]))))
print("Ortogonalidad V:", np.max(np.abs(V.T @ V - np.eye(V.shape[1]))))
print("Reconstrucción:", np.max(np.abs(X_train - U @ np.diag(S) @ V.T)))

In [ ]:

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def confusion_matrix_simple(W, X_val, Y_val):
    # 1. Predicciones
    Y_pred = Wsvd @ X_val
    y_pred = np.argmax(Y_pred, axis=0)
    y_true = np.argmax(Y_val, axis=0)

    # 2. Matriz de confusión
    M = np.zeros((2,2), dtype=int)
    for t,p in zip(y_true, y_pred):
        M[t,p] += 1

    # 3. Accuracy
    acc = np.trace(M) / np.sum(M)

    return M, acc

M, acc = confusion_matrix_simple(Wsvd, X_val, Y_val)
print("Matriz de confusión:\n", M)
print("Accuracy:", acc)


sns.heatmap(M, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión')
plt.show()

In [23]:
def prodPunto(v, w):
    """
    Asume que el vector es columna
    """

    return np.sum(v*w)

In [8]:
import alc
def  normaExacta(A,p=[1,'inf']):
    res = []
    B = A.T
    if p == 1:
        for fila in B:
            suma = 0
            for valor in fila:
                suma+= abs(valor)
            res.append(suma)
    elif p == 'inf':
        for fila in A:
            suma = 0
            for valor in fila:
                suma+= abs(valor)
            res.append(suma)
    else:
        return None
    return max(res)

# Test L06-metpot2k, Aval

import numpy as np

#### TESTEOS
# Tests metpot2k

S = np.vstack([
    np.array([2,1,0])/np.sqrt(5),
    np.array([-1,2,5])/np.sqrt(30),
    np.array([1,-2,1])/np.sqrt(6)
              ]).T

# Pedimos que pase el 95% de los casos
exitos = 0
for i in range(100):
    D = np.diag(np.random.random(3)+1)*100
    A = S@D@S.T
    v,l,_ = alc.metpot2k(A,1e-15,1e5)
    if np.abs(l - np.max(D))< 1e-8:
        exitos += 1
assert exitos > 95


#Test con HH
exitos = 0
for i in range(100):
    v = np.random.rand(9)
    #v = np.abs(v)
    #v = (-1) * v
    ixv = np.argsort(-np.abs(v))
    D = np.diag(v[ixv])
    I = np.eye(9)
    H = I - 2*np.outer(v.T, v)/(np.linalg.norm(v)**2)   #matriz de HouseHolder

    A = H@D@H.T
    v,l,_ = alc.metpot2k(A, 1e-15, 1e5)
    #max_eigen = abs(D[0][0])
    if abs(l - D[0,0]) < 1e-8:         
        exitos +=1
assert exitos > 95



# Tests diagRH
D = np.diag([1,0.5,0.25])
S = np.vstack([
    np.array([1,-1,1])/np.sqrt(3),
    np.array([1,1,0])/np.sqrt(2),
    np.array([1,-1,-2])/np.sqrt(6)
              ]).T

A = S@D@S.T
SRH,DRH = alc.diagRH(A,tol=1e-15,K=1e5)
assert np.allclose(D,DRH)
assert np.allclose(np.abs(S.T@SRH),np.eye(A.shape[0]),atol=1e-7)



# Pedimos que pase el 95% de los casos
exitos = 0
for i in range(100):
    A = np.random.random((5,5))
    A = 0.5*(A+A.T)
    S,D = alc.diagRH(A,tol=1e-15,K=1e5)
    ARH = S@D@S.T
    e = normaExacta(ARH-A,p='inf')
    if e < 1e-5: 
        exitos += 1
assert exitos >= 95


In [25]:
def metpot2k(A, tol=1e-15, K=150):
    N = A.shape[0]

    # Si la matriz es nula, entonces todos los autovalores son 0
    if norma(A, 2) < tol:
        v = np.random.rand(N)
        v = v / norma(v, 2)
        v = np.expand_dims(v, axis=0).T
        return v, 0, 0

    # Vector inicial
    v = np.random.rand(N)
    v = v / norma(v, 2)

    # Primera transformación lineal
    v_monio = aplicTrans(A, v)
    v_monio = v_monio / norma(v_monio, 2)

    e = prodPunto(v_monio, v)
    k = 0
    
    # Iteración hasta que converja o k llegue al límite
    while abs(e - 1) > tol and k < K:

        v = v_monio

        # Aplico transformación lineal
        w = aplicTrans(A, v)
        w = w / norma(w, 2)

        v_monio = aplicTrans(A, w)
        v_monio = v_monio / norma(v_monio, 2)

        e = prodPunto(v_monio, v)
        k += 1

    # Calculo el autovalor
    v_monio1 = np.expand_dims(v_monio, axis=0).T
    lambd = aplicTrans(prodMat(v_monio1.T, A), v_monio).item()

    return v_monio, lambd, e-1

In [26]:
def diagRH(A, tol=1e-15, K=150):
    """
    Diagonalización recursiva con Householder
    (VERSIÓN CORREGIDA – SOLO MATRICES SIMÉTRICAS)

    Args:
        A: matriz simétrica NxN
        tol: tolerancia
        K: iteraciones máximas método potencia
    """

    # --- Seguridad: matriz cuadrada ---
    if A.shape[0] != A.shape[1]:
        print("Matriz no cuadrada")
        return None, None

    # --- Seguridad: matriz simétrica ---
    if not np.allclose(A, A.T, atol=1e-12):
        print("ERROR: diagRH requiere matriz simétrica")
        return None, None

    N = A.shape[0]

    # --- Caso base ---
    if N == 1:
        return np.eye(1), A.copy()

    # =====================================================
    # 1) AUTOVECTOR / AUTOVALOR DOMINANTE (NORMALIZADO)
    # =====================================================
    v1, lambda1, _ = metpot2k(A, tol, K)

    # Asegurar normalización consistente
    v1 = v1.reshape((N, 1))
    nv = norma(np.squeeze(v1), 2)
    if nv < 1e-14:
        print("Error: autovector nulo en método potencia")
        return None, None
    v1 = v1 / nv

    # Forzar signo para estabilidad
    if v1[0,0] < 0:
        v1 = -v1

    # =====================================================
    # 2) HOUSEHOLDER: LLEVAR v1 → e1
    # =====================================================
    e1 = np.zeros((N,1))
    e1[0,0] = 1

    u = v1 - e1
    nu = norma(np.squeeze(u), 2)

    if nu < 1e-14:
        Hv1 = np.eye(N)
    else:
        factor = 2.0 / (nu * nu)
        Hv1 = np.eye(N) - factor * (u @ u.T)

    # =====================================================
    # 3) A ← H A H^T  (bloque superior queda λ₁)
    # =====================================================
    A[:] = prodMat(prodMat(Hv1, A), Hv1.T)

    # =====================================================
    # 4) EXTRAER submatriz Ahat = A[1:,1:]
    # =====================================================
    Ahat = A[1:, 1:].copy()   # COPIA → ¡NO VISTA!

    # =====================================================
    # 5) LLAMADA RECURSIVA
    # =====================================================
    Shat, Dhat = diagRH(Ahat, tol, K)
    if Shat is None:
        return None, None

    # =====================================================
    # 6) ARMAR MATRIZ D COMPLETA
    # =====================================================
    D = np.zeros((N,N))
    D[0,0] = lambda1
    D[1:,1:] = Dhat

    # =====================================================
    # 7) EXTENDER SHAT
    # =====================================================
    Hprod = np.zeros((N,N))
    Hprod[0,0] = 1
    Hprod[1:,1:] = Shat

    # =====================================================
    # 8) MATRIZ FINAL S
    # =====================================================
    S = prodMat(Hv1, Hprod)

    return S, D


In [ ]:
import numpy as np
import alc

def test_diagRH_debug(A, nombre=""):
    print(f"\n===== {nombre} =====")
    print("A =\n", A)

    try:
        S, D = alc.diagRH(A)

        if S is None:
            print("diagRH devolvió None (matriz no válida para este método).")
            return

        print("\n--- OUTPUT ---")
        print("Autovalores (diag D) =", np.diag(D))

        print("\nMatriz S (autovectores) =")
        print(S)

        # --- Checks ---
        print("\nError reconstrucción ||A - S D S^T|| =",
              np.linalg.norm(A - S @ D @ S.T))

        print("Check: S ortonormal:",
              np.allclose(S.T @ S, np.eye(A.shape[0]), atol=1e-8))

        print("Check: A ≈ S D S^T:",
              np.allclose(A, S @ D @ S.T, atol=1e-8))

    except Exception as e:
        print("ERROR:", e)


def test_diagRH():
    print("=== TEST diagRH ===")

    # 1) Caso NO simétrico → debe fallar
    A = np.array([[1., 2., 3.],
                  [0., 4., 5.],
                  [0., 0., 6.]])
    test_diagRH_debug(A, "Caso 1: NO ES SIMÉTRICA")

    # 2) Matriz simétrica aleatoria 5x5
    M = np.random.randn(5,5)
    A = (M + M.T)/2
    test_diagRH_debug(A, "Caso 2: Simétrica aleatoria 5x5")

    # 3) Matriz simétrica singular
    A = np.array([[1., 2.],
                  [2., 4.]])
    test_diagRH_debug(A, "Caso 3: Simétrica singular")

    # 4) Matriz simétrica grande
    M = np.random.randn(40,40)
    A = (M + M.T)/2
    test_diagRH_debug(A, "Caso 4: Simétrica 40x40")

test_diagRH()


In [12]:
# Tests L08
# Matrices al azar
import numpy as np
import alc

def genera_matriz_para_test(m,n=2,tam_nucleo=0):
    if tam_nucleo == 0:
        A = np.random.random((m,n))
    else:
        A = np.random.random((m,tam_nucleo))
        A = np.hstack([A,A])
    return(A)

def test_svd_reducida_mn(A,tol=1e-15):
    m,n = A.shape
    hU,hS,hV = alc.svd_reducida(A,tol=tol)
    nU,nS,nVT = np.linalg.svd(A)
    r = len(hS)+1
    assert np.all(np.abs(np.abs(np.diag(hU.T @ nU))-1)<10**r*tol), 'Revisar calculo de hat U en ' + str((m,n))
    assert np.all(np.abs(np.abs(np.diag(nVT @ hV))-1)<10**r*tol), 'Revisar calculo de hat V en ' + str((m,n))
    assert len(hS) == len(nS[np.abs(nS)>tol]), 'Hay cantidades distintas de valores singulares en ' + str((m,n))
    assert np.all(np.abs(hS-nS[np.abs(nS)>tol])<10**r*tol), 'Hay diferencias en los valores singulares en ' + str((m,n))

for m in [2,5,10,20]:
    for n in [2,5,10,20]:
        for _ in range(10):
            A = genera_matriz_para_test(m,n)
            test_svd_reducida_mn(A)


# Matrices con nucleo

m = 12
for tam_nucleo in [2,4,6]:
    for _ in range(10):
        A = genera_matriz_para_test(m,tam_nucleo=tam_nucleo)
        test_svd_reducida_mn(A)

# Tamaños de las reducidas
A = np.random.random((8,6))
for k in [1,3,5]:
    hU,hS,hV = alc.svd_reducida(A,k=k)
    assert hU.shape[0] == A.shape[0], 'Dimensiones de hU incorrectas (caso a)'
    assert hV.shape[0] == A.shape[1], 'Dimensiones de hV incorrectas(caso a)'
    assert hU.shape[1] == k, 'Dimensiones de hU incorrectas (caso a)'
    assert hV.shape[1] == k, 'Dimensiones de hV incorrectas(caso a)'
    assert len(hS) == k, 'Tamaño de hS incorrecto'


In [ ]:
import numpy as np
# TESTS CON FUNCIONES NP


def normaExacta(A,p=[1,'inf']):
    res = []
    B = A.T
    if p == 1:
        for fila in B:
            suma = 0
            for valor in fila:
                suma+= abs(valor)
            res.append(suma)
    elif p == 'inf':
        for fila in A:
            suma = 0
            for valor in fila:
                suma+= abs(valor)
            res.append(suma)
    else:
        return None
    return max(res)
#### TESTEOS
# Tests metpot2k

S = np.vstack([
    np.array([2,1,0])/np.sqrt(5),
    np.array([-1,2,5])/np.sqrt(30),
    np.array([1,-2,1])/np.sqrt(6)
              ]).T

# Pedimos que pase el 95% de los casos
exitos = 0
for i in range(100):
    D = np.diag(np.random.random(3)+1)*100
    A = S@D@S.T
    v,l,_ = alc.metpot2k_np(A,1e-15,1e5)
    if np.abs(l - np.max(D))< 1e-8:
        exitos += 1
assert exitos > 95


#Test con HH
exitos = 0
for i in range(100):
    v = np.random.rand(9)
    #v = np.abs(v)
    #v = (-1) * v
    ixv = np.argsort(-np.abs(v))
    D = np.diag(v[ixv])
    I = np.eye(9)
    H = I - 2*np.outer(v.T, v)/(np.linalg.norm(v)**2)   #matriz de HouseHolder

    A = H@D@H.T
    v,l,_ = alc.metpot2k_np(A, 1e-15, 1e5)
    #max_eigen = abs(D[0][0])
    if abs(l - D[0,0]) < 1e-8:         
        exitos +=1
assert exitos > 95



# Tests diagRH
D = np.diag([1,0.5,0.25])
S = np.vstack([
    np.array([1,-1,1])/np.sqrt(3),
    np.array([1,1,0])/np.sqrt(2),
    np.array([1,-1,-2])/np.sqrt(6)
              ]).T

A = S@D@S.T
SRH,DRH = alc.diagRH_np(A,tol=1e-15,K=1e5)
assert np.allclose(D,DRH)
assert np.allclose(np.abs(S.T@SRH),np.eye(A.shape[0]),atol=1e-7)



# Pedimos que pase el 95% de los casos
exitos = 0
for i in range(100):
    A = np.random.random((5,5))
    A = 0.5*(A+A.T)
    S,D = alc.diagRH_np(A,tol=1e-15,K=1000)
    ARH = S@D@S.T
    e = normaExacta(ARH-A,p='inf')
    if e < 1e-5: 
        exitos += 1
assert exitos >= 95